In [1]:
import pandas as pd
df = pd.read_csv('archive/Korea Income and Welfare.csv')
df.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10


In [2]:
df.describe()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion
count,9.285700e+04,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000
mean,4.365181e+07,2011.517958,7.517958,3.691353,3441.122327,2.484304,1.296316,1952.956740,4.511895,1.679389,1.499521
std,2.619007e+07,4.011222,4.011222,2.058961,4176.324522,1.292251,0.456634,16.006065,1.669887,1.103413,0.531572
min,1.010100e+04,2005.000000,1.000000,1.000000,-232174.000000,1.000000,1.000000,1910.000000,2.000000,0.000000,1.000000
25%,2.177120e+07,2008.000000,4.000000,2.000000,1140.000000,1.000000,1.000000,1939.000000,3.000000,1.000000,1.000000
50%,4.322010e+07,2012.000000,8.000000,3.000000,2428.000000,2.000000,1.000000,1952.000000,5.000000,1.000000,1.000000
75%,6.190030e+07,2015.000000,11.000000,5.000000,4695.000000,3.000000,2.000000,1966.000000,5.000000,2.000000,2.000000
max,9.800070e+07,2018.000000,14.000000,7.000000,468209.000000,9.000000,2.000000,2002.000000,9.000000,9.000000,9.000000


In [3]:
df.isnull().sum()

id                    0
year                  0
wave                  0
region                0
income                0
family_member         0
gender                0
year_born             0
education_level       0
marriage              0
religion              0
occupation            0
company_size          0
reason_none_worker    0
dtype: int64

In [4]:
avg_income_region = df.groupby('region')['income'].mean()
print(avg_income_region)

region
1    4028.787273
2    3907.235626
3    3433.993306
4    2771.124671
5    3742.423781
6    3081.881088
7    2870.977443
Name: income, dtype: float64


In [5]:
family_member_education = df.groupby('education_level')['family_member'].sum()
print(family_member_education)

education_level
2    15546
3    40277
4    28743
5    77809
6    18074
7    42382
8     7094
9      760
Name: family_member, dtype: int64


In [6]:
gender_religion = df.groupby(['gender', 'religion'])['marriage'].value_counts(normalize=True)
print(gender_religion)

gender  religion  marriage
1       1         1           0.887081
                  5           0.037929
                  3           0.036693
                  2           0.032549
                  4           0.005080
                  6           0.000568
                  0           0.000067
                  9           0.000033
        2         1           0.814362
                  5           0.073169
                  3           0.062030
                  2           0.040260
                  4           0.009217
                  6           0.000679
                  0           0.000226
                  9           0.000057
        9         1           0.875000
                  2           0.041667
                  5           0.041667
                  0           0.020833
                  6           0.020833
2       1         2           0.712944
                  3           0.151356
                  1           0.062563
                  5           0.04743

In [7]:
q3 = df.quantile(0.75, numeric_only=True)
q1 = df.quantile(0.25, numeric_only=True)

iqr = q3 - q1

def is_outlier(df):
    score = df['income']
    if score > q3['income'] + 1.5*iqr['income'] or score < q1['income'] - 1.5 * iqr['income']:
        return True
    else:
        return False

df['income_outlier'] = df.apply(is_outlier, axis = 1)

df = df.loc[df['income_outlier'] == False]

del df['income_outlier']

df['income'].value_counts()

C:\Users\wjsdl\AppData\Local\Temp\ipykernel_5164\2081928226.py:1: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  q3 = df.quantile(0.75)
C:\Users\wjsdl\AppData\Local\Temp\ipykernel_5164\2081928226.py:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  q1 = df.quantile(0.25)


3000.0    71
2400.0    70
1800.0    69
1200.0    57
840.0     56
          ..
4130.4     1
6836.0     1
3354.4     1
3706.2     1
1446.3     1
Name: income, Length: 10336, dtype: int64

3000.0    71
2400.0    70
1800.0    69
1200.0    57
840.0     56
          ..
4130.4     1
6836.0     1
3354.4     1
3706.2     1
1446.3     1
Name: income, Length: 10336, dtype: int64

In [11]:
df_job_on = df[(df['occupation'] != ' ') & (df['reason_none_worker'] == ' ') ]

In [12]:
df_job_on = df_job_on[(df_job_on['occupation'] != ' ') & (df_job_on['company_size'] != ' ') ]

In [13]:
df_job_off = df[(df['occupation'] == ' ') & (df['reason_none_worker'] != ' ') ]

df_job_on.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
8,20101,2005,1,1,1257.0,1,2,1945,4,2,2,421,1,
10,20101,2007,3,1,602.0,1,2,1945,4,2,2,411,1,
11,20101,2008,4,1,1972.0,1,2,1945,4,2,2,951,1,
12,20101,2009,5,1,1638.0,1,2,1945,4,2,1,951,1,
13,20101,2010,6,1,1598.0,1,2,1945,4,2,2,951,1,


In [30]:
df_soldier = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')]
df_soldier.loc[:, 'occupation'] = '100'

In [31]:
soldier_num = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')].index
df_job_on.drop(soldier_num, inplace=True)

In [32]:
df_job_on['occupation'] = df_job_on['occupation'].str[0] * 2

df_job_on = pd.concat([df_job_on,df_soldier])

In [33]:
df_job_on['occupation'].value_counts()

66    11248
99    10861
88     6961
77     5927
33     5535
22     4785
55     4351
44     3820
11     2066
Name: occupation, dtype: int64

In [34]:
df_job_off['reason_none_worker'].replace('11', '0', inplace = True)

In [35]:
num = df_job_off[df_job_off['reason_none_worker']=='99'].index

df_job_off.drop(num, inplace=True)

In [36]:
df_job_off['reason_none_worker'].value_counts()

10    18988
8      4263
9      2668
1      2484
7      1117
6      1065
4       703
5       323
0       235
3        52
2         7
Name: reason_none_worker, dtype: int64

In [37]:
df.drop(['id','year'], axis = 1, inplace = True)

KeyError: "['id', 'year'] not found in axis"

In [38]:
df = df[df['income'] >= 0]

In [39]:
num1 = df[(df['marriage'] == 0) | (df['marriage'] == 9)].index
df.drop(num1, inplace =True)

In [40]:
num2 = df[df['religion'] == 9].index
df.drop(num2, inplace=True)

In [41]:
num3 = df[df['company_size']=='99'].index
df.drop(num3, inplace=True)

In [42]:
print(df['region'].value_counts())
print(df['gender'].value_counts())
print(df['education_level'].value_counts())
print(df['marriage'].value_counts())
print(df['religion'].value_counts())

2    18442
7    15433
3    15412
1    13537
4    11861
5     7376
6     6709
Name: region, dtype: int64
1    61401
2    27369
Name: gender, dtype: int64
5    24921
3    21030
7    12327
4    12059
2    10829
6     5564
8     1844
9      196
Name: education_level, dtype: int64
1    53315
2    21154
3     7387
5     5716
4     1144
6       54
Name: marriage, dtype: int64
1    44856
2    43914
Name: religion, dtype: int64


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88770 entries, 0 to 92856
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   wave                88770 non-null  int64  
 1   region              88770 non-null  int64  
 2   income              88770 non-null  float64
 3   family_member       88770 non-null  int64  
 4   gender              88770 non-null  int64  
 5   year_born           88770 non-null  int64  
 6   education_level     88770 non-null  int64  
 7   marriage            88770 non-null  int64  
 8   religion            88770 non-null  int64  
 9   occupation          88770 non-null  object 
 10  company_size        88770 non-null  object 
 11  reason_none_worker  88770 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 8.8+ MB
